In [12]:
!pip install arxiv

In [2]:
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper

In [5]:
api_wrapper = WikipediaAPIWrapper(top_k_results = 1, doc_content_chars_max = 200)
wiki = WikipediaQueryRun(api_wrapper = api_wrapper)

In [6]:
wiki.name

'wikipedia'

In [13]:
from langchain_community.tools.arxiv.tool import ArxivQueryRun
from langchain_community.utilities.arxiv import ArxivAPIWrapper

arxiv_api = ArxivAPIWrapper(top_k_results=3)
arxiv = ArxivQueryRun(api_wrapper=arxiv_api)

In [14]:
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import OllamaEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

# Load the webpage
loader = WebBaseLoader("https://docs.smith.langchain.com/")
docs = loader.load()

# Split the content into chunks
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
documents = text_splitter.split_documents(docs)

# Use Ollama embeddings (make sure Ollama is running and gemma:2b is pulled)
embeddings = OllamaEmbeddings(model="gemma:2b")

# Create the FAISS vector store
vectordb = FAISS.from_documents(documents, embeddings)

# Get a retriever from the vector store
retriever = vectordb.as_retriever()

retriever


VectorStoreRetriever(tags=['FAISS', 'OllamaEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x10cc37550>, search_kwargs={})

In [15]:
from langchain.tools.retriever import create_retriever_tool
retriever_tool=create_retriever_tool(retriever,"langsmith_search",
					  "Search for information about LangSmith. For any questions about LangSmith, you must use this tool!")

In [16]:
retriever_tool.name

'langsmith_search'

In [17]:
tools=[wiki,arxiv,retriever_tool]

In [18]:
tools

[WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper(wiki_client=<module 'wikipedia' from '/Users/jamil.226/Desktop/Jamil/University/COMSATS/AI Training Summer/08. Langchain/8. Agents/venv/lib/python3.10/site-packages/wikipedia/__init__.py'>, top_k_results=1, lang='en', load_all_available_meta=False, doc_content_chars_max=200)),
 ArxivQueryRun(api_wrapper=ArxivAPIWrapper(arxiv_search=<class 'arxiv.Search'>, arxiv_exceptions=(<class 'arxiv.ArxivError'>, <class 'arxiv.UnexpectedEmptyPageError'>, <class 'arxiv.HTTPError'>), top_k_results=3, ARXIV_MAX_QUERY_LENGTH=300, continue_on_failure=False, load_max_docs=100, load_all_available_meta=False, doc_content_chars_max=4000)),
 Tool(name='langsmith_search', description='Search for information about LangSmith. For any questions about LangSmith, you must use this tool!', args_schema=<class 'langchain_core.tools.retriever.RetrieverInput'>, func=functools.partial(<function _get_relevant_documents at 0x10ce09750>, retriever=VectorStoreRetriever(tags

In [19]:
from langchain_community.chat_models import ChatOllama

llm = ChatOllama(model="gemma:2b")


/var/folders/mb/dm2cmb7s1t1fk_2rx8q97mq40000gn/T/ipykernel_48788/244168907.py:3: LangChainDeprecationWarning: The class `ChatOllama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import ChatOllama``.
  llm = ChatOllama(model="gemma:2b")


In [20]:
from langchain import hub
# Get the prompt to use - you can modify this!
prompt = hub.pull("hwchase17/openai-functions-agent")
prompt.messages

/Users/jamil.226/Desktop/Jamil/University/COMSATS/AI Training Summer/08. Langchain/8. Agents/venv/lib/python3.10/site-packages/langsmith/client.py:272: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='You are a helpful assistant'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='chat_history', optional=True),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='{input}'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='agent_scratchpad')]

In [26]:
### Agents
from langchain.agents import create_openai_tools_agent
agent=create_openai_tools_agent(llm,tools,prompt)

In [22]:
## Agent Executer
from langchain.agents import AgentExecutor
agent_executor=AgentExecutor(agent=agent,tools=tools,verbose=True)
agent_executor

AgentExecutor(verbose=True, agent=RunnableMultiActionAgent(runnable=RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: format_to_openai_tool_messages(x['intermediate_steps']))
})
| ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], optional_variables=['chat_history'], input_types={'chat_history': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag

In [23]:
agent_executor.invoke({"input":"Tell me about Langsmith"})



> Entering new AgentExecutor chain...
Sure. Here's a summary of Langsmith:

**Langsmith** is a **character in Final Fantasy VII**, a role-playing video game released for PlayStation and the PlayStation 2. The character is a **smith** with a deep connection to **lost technology** and a strong desire to **restore them**.

**Key characteristics:**

* **Weapon:** Smith's forge is his primary weapon and source of crafting.
* **Personality:** Friendly and welcoming, but also wary and guarded around his age.
* **Background:** Langsmith was a young blacksmith in **Sephiroth** when it fell under **Sephiroth's control**. He was captured and experimented on, leading to the creation of the **Sephiroth Armor** around his body.
* **Role in Final Fantasy VII:** Langsmith is a key member of the **Black Mage's Circle** and aids the party in acquiring lost technology. He is also a key figure in **reopening Sephiroth's laboratory** and restoring the **Sephiroth Armor** to its former glory.

**Additiona

{'input': 'Tell me about Langsmith',
 'output': "Sure. Here's a summary of Langsmith:\n\n**Langsmith** is a **character in Final Fantasy VII**, a role-playing video game released for PlayStation and the PlayStation 2. The character is a **smith** with a deep connection to **lost technology** and a strong desire to **restore them**.\n\n**Key characteristics:**\n\n* **Weapon:** Smith's forge is his primary weapon and source of crafting.\n* **Personality:** Friendly and welcoming, but also wary and guarded around his age.\n* **Background:** Langsmith was a young blacksmith in **Sephiroth** when it fell under **Sephiroth's control**. He was captured and experimented on, leading to the creation of the **Sephiroth Armor** around his body.\n* **Role in Final Fantasy VII:** Langsmith is a key member of the **Black Mage's Circle** and aids the party in acquiring lost technology. He is also a key figure in **reopening Sephiroth's laboratory** and restoring the **Sephiroth Armor** to its former g

In [24]:
agent_executor.invoke({"input":"What's the paper 1605.08386 about?"})



> Entering new AgentExecutor chain...
I am unable to access external websites or specific research databases, so I cannot provide a summary of the paper you requested.

> Finished chain.


{'input': "What's the paper 1605.08386 about?",
 'output': 'I am unable to access external websites or specific research databases, so I cannot provide a summary of the paper you requested.'}